- https://arxiv.org/abs/1802.00308; https://medium.com/intuition/deep-recurrent-neural-networks-for-electroencephalography-analysis-7c428c50f038

- https://github.com/Khalizo/Deep-Learning-Detection-Of-EEG-Based-Attention/tree/master

- https://github.com/SuperBruceJia/EEG-DL

- https://pmc.ncbi.nlm.nih.gov/articles/PMC10372445/

- https://www.sciencedirect.com/science/article/pii/B9780128240540000095:

    More channels; C;ass imbalance; CNN, Hybrid R-CNN, DT, NN, SVM, KNN

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os

In [19]:
df = pd.read_csv('data/highAHI/OSAA10002_merged.csv')

In [20]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['Hypopnea_Label_Encode'])

# Check mapping:
print(dict(zip(le.classes_, le.transform(le.classes_))))
# Example output:
# {'-1': 0, 'M': 1, 'P': 2, 'PP': 3}


{'-1': np.int64(0), 'M': np.int64(1), 'P': np.int64(2), 'PP': np.int64(3)}


In [21]:
df.Hypopnea_Label_Encode.value_counts()

Hypopnea_Label_Encode
-1    2519
P       51
M       51
PP      46
Name: count, dtype: int64

In [7]:
df

,Mini_Epoch_Index,Sleeping_stage,Delta,Theta,Alpha,Sigma,Beta,Gamma,TotalAbsPow,Hypopnea_Label_Encode
0,1,0,0.353299,0.084716,0.290772,0.048878,0.125810,0.096525,8.252734e-11,-1
1,2,-1,0.350105,0.054575,0.354590,0.040920,0.121198,0.078611,6.210068e-11,-1
2,3,-1,0.247155,0.066444,0.499360,0.027917,0.093214,0.065909,6.417000e-11,-1
3,4,0,0.354059,0.105969,0.223098,0.069373,0.162833,0.084667,4.133262e-11,-1
4,5,0,0.118186,0.041903,0.747283,0.015630,0.052638,0.024361,1.324671e-10,-1
...,...,...,...,...,...,...,...,...,...,...
2662,2663,0,0.584954,0.097088,0.032402,0.082907,0.118319,0.084329,8.745815e-11,-1
2663,2664,0,0.693394,0.046640,0.019170,0.032831,0.113093,0.094872,2.506265e-10,-1
2664,2665,0,0.549748,0.107757,0.029323,0.039583,0.146618,0.126971,1.005049e-10,-1
2665,2666,-1,0.593619,0.078667,0.063104,0.031854,0.125769,0.106987,1.174124e-10,-1


In [22]:
X = []
y = []

features = ["Sleeping_stage","Delta","Theta","Alpha","Sigma","Beta","Gamma","TotalAbsPow"]

for i in range(2, len(df)):
    seq = df.iloc[i-2:i][features].values
    label = df.iloc[i]['label_encoded']
    
    X.append(seq)
    y.append(label)

X = np.array(X)
y = np.array(y)


### does pp, p, m, -1 have any statistical differences, - median, mean of delta, alpha, beta, gamma etc (10 seconds epochs)
### does rem, 1, 2, 3, wake have any statistical differences, - median, mean of delta, alpha, beta, gamma etc (10 seconds epochs)
### take out all wake ones - -1 and look at the means again
### every second - then use rnn - 2560 samples - label it as stage of sleep

In [23]:
X.shape, X[0], X[1]

((2665, 2, 8),
 array([[ 0.00000000e+00,  3.53298728e-01,  8.47159635e-02,
          2.90772044e-01,  4.88784679e-02,  1.25810049e-01,
          9.65247484e-02,  8.25273367e-11],
        [-1.00000000e+00,  3.50104641e-01,  5.45749447e-02,
          3.54590414e-01,  4.09202009e-02,  1.21198380e-01,
          7.86114194e-02,  6.21006795e-11]]),
 array([[-1.00000000e+00,  3.50104641e-01,  5.45749447e-02,
          3.54590414e-01,  4.09202009e-02,  1.21198380e-01,
          7.86114194e-02,  6.21006795e-11],
        [-1.00000000e+00,  2.47155461e-01,  6.64443417e-02,
          4.99359931e-01,  2.79174666e-02,  9.32140987e-02,
          6.59087010e-02,  6.41699998e-11]]))

In [24]:
y.shape, y[0], y[1]

((2665,), np.int64(0), np.int64(0))

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)


In [33]:
import torch
from torch.utils.data import Dataset, DataLoader

class HypopneaDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

train_ds = HypopneaDataset(X_train, y_train)
test_ds = HypopneaDataset(X_test, y_test)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32)


In [38]:
y.shape, y[0]  # (2, num_features)     

((2665,), np.int64(0))

In [37]:
X.shape, X[0]  # (2, num_features)     

((2665, 2, 8),
 array([[ 0.00000000e+00,  3.53298728e-01,  8.47159635e-02,
          2.90772044e-01,  4.88784679e-02,  1.25810049e-01,
          9.65247484e-02,  8.25273367e-11],
        [-1.00000000e+00,  3.50104641e-01,  5.45749447e-02,
          3.54590414e-01,  4.09202009e-02,  1.21198380e-01,
          7.86114194e-02,  6.21006795e-11]]))

```
Input:     (batch_size, seq_len=2, input_size=8)
             │
             ▼
         ┌─────────────┐
         │   RNN Layer │
         │  (8 → 32)   │
         └─────────────┘
             │
   Output shape: (batch_size, seq_len=2, hidden_size=32)
             │
             ▼
Take the last timestep output:
    out[:, -1, :] → shape = (batch_size, hidden_size=32)
             │
             ▼
     ┌────────────────────┐
     │  Fully Connected   │
     │   (Linear 32 → 4)  │
     └────────────────────┘
             │
             ▼
Final Output: (batch_size, num_classes=4)
→ Apply CrossEntropyLoss for training
```

In [39]:
import torch.nn as nn

class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(RNNClassifier, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.rnn(x)  # out: (batch, seq_len, hidden)
        out = out[:, -1, :]   # take last timestep output
        out = self.fc(out)
        return out

model = RNNClassifier(input_size=8, hidden_size=32, num_classes=4)


In [40]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


Epoch 1, Loss: 48.1950
Epoch 2, Loss: 19.1318
Epoch 3, Loss: 17.8343
Epoch 4, Loss: 17.3154
Epoch 5, Loss: 17.2825
Epoch 6, Loss: 17.0805
Epoch 7, Loss: 17.2770
Epoch 8, Loss: 17.1630
Epoch 9, Loss: 17.0212
Epoch 10, Loss: 17.0836


In [41]:
model.eval()
correct, total = 0, 0

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 94.56%
